In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.common.by import By
import pandas as pd
import numpy as np

In [ ]:
url = "https://app.cbtm.org.br/UI/Site/RatingResultado"

In [ ]:
firefox = webdriver.Firefox()
firefox.get(url)

In [ ]:
ratingElem = firefox.find_element_by_xpath('//*[@id="cmbRating"]')
ratingElem.click()
#ratingList = firefox.find_elements_by_xpath('//*[@id="cmbRating_DDD_PW-1"]')

In [ ]:
#Corrigir bug: Nâo precisar mover o cursor na tabela para pegar todos os rating ao invés de só o "H"

In [ ]:
ratingList = firefox.find_element_by_xpath('//*[@id="cmbRating_DDD_PW-1"]')
ratingElements = ratingList.find_elements_by_class_name('dxeListBoxItem_MetropolisBlue')
ratingIDs = []
ratingNames = []
for rating in ratingElements:
    ratingIDs.append(rating.get_attribute('id'))
    ratingNames.append(rating.text)

In [ ]:
dfs = []
columnNames = ['Ranking', 'Pontos', 'Nome', 'Clube', 'ID detalhes']
for _id in ratingIDs:
    ratingList = firefox.find_element_by_xpath('//*[@id="cmbRating_DDD_PW-1"]')
    firefox.find_element_by_id(_id).click()
    table = WebDriverWait(firefox, 60).until(expected_conditions.presence_of_element_located((By.XPATH, '//*[@id="grid_DXMainTable"]')))
    #table = firefox.find_element_by_xpath('//*[@id="grid_DXMainTable"]')
    data = np.frompyfunc(lambda x:x.text, 1, 1)(table.find_elements_by_tag_name('span'))
    dataReshape = data.reshape(-1,5)
    df = pd.DataFrame(dataReshape)
    df.columns = columnNames
    dfs.append(df)
    ratingElem = firefox.find_element_by_xpath('//*[@id="cmbRating"]')
    ratingElem.click()

In [ ]:
nRatFem = 10
nRatMasc = 14

for i in range(nRatFem):
    dfs[i]['Rating'] = chr(i + ord('A'))
    dfs[i]['Categoria'] = 'F'
for i in range(nRatFem, nRatFem+nRatMasc):
    dfs[i]['Rating'] = chr(i - nRatFem + ord('A'))
    dfs[i]['Categoria'] = 'M'
    if (i >= 20):
        dfs[i]['Rating'] = chr(i + 1 - nRatFem + ord('A')) #Sem rating K
        
df_unico = pd.concat(dfs, ignore_index = True)
df_unico['Pontos'] = df_unico['Pontos'].str.replace(r'\D', '').astype(int)
df_unico['Ranking'] = df_unico['Ranking'].str.replace(r'\D', '').astype(int)

In [ ]:
df_unico.to_csv('rating_CBTM.csv')